Players Data Scraping

In [1]:
import requests
import json
import pandas as pd
import os

In [2]:
season_names = ['2014', '2015', '2016', '2017', '2018', '2019']
gws = ['3', '5', '10']
leagues = ['EPL', 'La_liga', 'Bundesliga', 'Serie_A', 'Ligue_1']

In [3]:
def scrape_understat(payload):
    #Build request using url, headers (mimicking what Firefox does normally)
    #Works best with verify=True as you won't get the ssl errors. Payload is 
    #taylored for each request
    url = 'https://understat.com/main/getPlayersStats/'
    headers = {'content-type':'application/json; charset=utf-8',
    'Host': 'understat.com',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate, br',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Length': '39',
    'Origin': 'https: // understat.com',
    'Connection': 'keep - alive',
    'Referer': 'https: // understat.com / league / EPL'
    }
    response = requests.post(url, data=payload, headers = headers, verify=True)
    response_json = response.json()
    inner_wrapper = response_json['response']
    json_player_data = inner_wrapper['players']
    return json_player_data

In [4]:
def clean_df(player_df, weeks):
    # Get rid of the columns that we don't care about
    #player_df.drop(['yellow_cards','red_cards', 'xGChain','xGBuildup','games','time'], axis=1, inplace=True)
    player_df  = player_df.rename(columns={'goals':'goals_'+weeks,'xG':'xG_'+weeks,'assists':'assists_'+weeks, 'xA':'xA_'+weeks, 'shots':'shots_'+weeks, 'key_passes':
        'key_passes_'+weeks,'npg':'npg_'+weeks,'npxG':'npxG_'+weeks})
    
    return(player_df)

In [5]:
def gw_data(season , league,  no_of_gw):
#     Create Pandas dataframes from each html table
    print('Getting data for last {} matches'.format(no_of_gw))
    json_player_data = scrape_understat({'league':'EPL', 'season':season, 'n_last_matches': no_of_gw})
    gw_table = pd.DataFrame(json_player_data)
    gw_df = clean_df(gw_table,'3wks')
    #Replace Position indentifiers with something more useful
    gw_df['position'] = gw_df['position'].str.slice(0,1)
    position_map = {'D':'DEF', 'F':'FWD', 'M':'MID', 'G':'GK', 'S':'FWD'}
    gw_df = gw_df.replace({'position': position_map})
    gw_df.to_csv(r'E:\AMotefaker\ABC\FPA\Web Scraping\Understat Data for Teams-Players (2014- present)\Data\Player_Data\gw_data\last_{}_gw_data.csv'.format(no_of_gw), encoding='utf-8', index=False)
    print('last {} matches csv data written'.format(no_of_gw))
    return gw_df

In [8]:
last_3_gw_data_EPL = gw_data(season_names[-1], leagues[0], gws[0])
last_5_gw_data_EPL = gw_data(season_names[-1], leagues[0], gws[1])
last_10_gw_data_EPL = gw_data(season_names[-1], leagues[0], gws[2])

Getting data for last 3 matches
last 3 matches csv data written
Getting data for last 5 matches
last 5 matches csv data written
Getting data for last 10 matches
last 10 matches csv data written


In [9]:
def season_data(season, league):
    print('Getting data for {} season'.format(season))
    json_player_data = scrape_understat({'league': league, 'season':season})
    season_table = pd.DataFrame(json_player_data)
    season_df = clean_df(season_table, 'season')
    season_df.to_csv(r'E:\AMotefaker\ABC\FPA\Web Scraping\Understat Data for Teams-Players (2014- present)\Data\Player_Data\season_data\{}_whole_season_data.csv'.format(season), encoding='utf-8', index=False)
    print('csv file for {} season written'.format(season))
    return season_df

In [15]:
season_1415 = season_data(season_names[0], leagues[0])
season_1516 = season_data(season_names[1], leagues[0])
season_1617 = season_data(season_names[2], leagues[0])
season_1718 = season_data(season_names[3], leagues[0])
season_1819 = season_data(season_names[4], leagues[0])
season_1920 = season_data(season_names[5], leagues[0])

Getting data for 2014 season
csv file for 2014 season written
Getting data for 2015 season
csv file for 2015 season written
Getting data for 2016 season
csv file for 2016 season written
Getting data for 2017 season
csv file for 2017 season written
Getting data for 2018 season
csv file for 2018 season written
Getting data for 2019 season
csv file for 2019 season written


In [17]:
season_1415

,id,player_name,games,time,goals_season,xG_season,assists_season,xA_season,shots_season,key_passes_season,yellow_cards,red_cards,position,team_title,npg_season,npxG_season,xGChain,xGBuildup
0,619,Sergio Agüero,33,2551,26,25.270159743726254,8,5.568922242149711,148,33,4,0,F S,Manchester City,21,20.70318364351988,27.805154908448458,6.878173082135618
1,647,Harry Kane,34,2589,21,17.15729223564267,4,3.922500966116786,112,27,4,0,F M S,Tottenham,19,14.873822528868914,16.488438992761075,5.549698735587299
2,802,Diego Costa,26,2111,20,15.219103761017323,3,4.554670915938914,76,41,8,0,F S,Chelsea,19,14.45793492347002,21.365789908915758,5.2769727278500795
3,848,Charlie Austin,35,3078,18,17.881850190460682,5,2.548747032880783,131,23,4,1,F,Queens Park Rangers,15,14.076042897999287,13.718276167288423,3.041321298107505
4,498,Alexis Sánchez,35,2967,16,13.451749975793064,8,8.49417957291007,122,82,4,0,F M S,Arsenal,16,12.690581078641117,27.15757153555751,10.736752955242991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,4528,Alejandro Faurlin,2,94,0,0.11255800724029541,0,0.4859839975833893,3,1,0,0,M S,Queens Park Rangers,0,0.11255800724029541,0.055191781371831894,0.022535664960741997
527,4530,Valentin Roberge,1,90,0,0.2825593948364258,0,0,1,0,1,0,D,Sunderland,0,0.2825593948364258,0,0
528,4531,Stuart O&#039;Keefe,2,88,0,0,0,0.019946111366152763,0,1,0,0,M S,Crystal Palace,0,0,0.019946111366152763,0.019946111366152763
529,4532,Gary Taylor-Fletcher,1,16,0,0,0,0,0,0,0,0,S,Leicester,0,0,0,0


In [18]:
season_1516

,id,player_name,games,time,goals_season,xG_season,assists_season,xA_season,shots_season,key_passes_season,yellow_cards,red_cards,position,team_title,npg_season,npxG_season,xGChain,xGBuildup
0,647,Harry Kane,38,3382,25,22.732073578983545,1,3.088511780835688,159,44,5,0,F,Tottenham,20,18.926266126334667,26.939671490341425,8.189033068716526
1,619,Sergio Agüero,30,2399,24,20.077213659882545,2,2.0287597198039293,119,27,1,0,F S,Manchester City,20,16.271369472146034,18.786789989098907,3.925958522595465
2,755,Jamie Vardy,36,3154,24,22.135407269001007,6,6.267152491956949,115,48,5,1,F,Leicester,19,17.56843115389347,24.595971267670393,2.8863560035824776
3,594,Romelu Lukaku,37,3194,18,20.05878111720085,6,5.6804345063865185,118,51,3,0,F S,Everton,17,18.53648015856743,24.024903159588575,7.63484107516706
4,750,Riyad Mahrez,37,3087,17,11.882148338481784,11,11.45409781858325,86,68,1,0,M S,Leicester,13,7.315135253593326,21.18506269901991,7.174407683312893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,1078,Sergi Canos,1,7,0,0,0,0,0,0,0,0,S,Liverpool,0,0,0,0
546,1079,Thomas Robson,1,90,0,0,0,0,0,0,0,0,D,Sunderland,0,0,0.09102849662303925,0.09102849662303925
547,1080,Rees Greenwood,1,53,0,0,0,0.02298414707183838,0,1,0,0,M,Sunderland,0,0,0.6708943247795105,0.6708943247795105
548,1084,Jonjoe Kenny,1,62,0,0.09364968538284302,0,0,1,0,0,0,S,Everton,0,0.09364968538284302,0.09364968538284302,0


In [19]:
season_1617

,id,player_name,games,time,goals_season,xG_season,assists_season,xA_season,shots_season,key_passes_season,yellow_cards,red_cards,position,team_title,npg_season,npxG_season,xGChain,xGBuildup
0,647,Harry Kane,30,2556,29,19.82009919732809,7,5.5538915153592825,110,41,3,0,F M S,Tottenham,24,15.253085978329182,21.94719305820763,4.12599990144372
1,594,Romelu Lukaku,37,3271,25,16.665452419780195,6,5.440816408023238,110,47,3,0,F S,Everton,24,15.904283582232893,21.251998490653932,3.9702013842761517
2,498,Alexis Sánchez,38,3233,24,17.724449135363102,10,8.988117071799934,129,78,6,0,F M S,Arsenal,22,15.44094256311655,27.14118772558868,9.08672859147191
3,619,Sergio Agüero,31,2408,20,22.672585003077984,3,4.665906261652708,139,31,4,1,F S,Manchester City,16,18.866740860044956,25.35206551104784,5.0266740852966905
4,802,Diego Costa,35,3101,20,15.431797001510859,7,4.912058966234326,111,42,10,0,F,Chelsea,20,14.670628163963556,22.003214471042156,6.565549122169614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,5575,Joel Pereira,1,90,0,0,0,0,0,0,0,0,GK,Manchester United,0,0,0.2117270529270172,0.2117270529270172
520,5576,James Husband,1,62,0,0,0,0,0,0,0,0,D,Middlesbrough,0,0,0.15693369507789612,0.15693369507789612
521,5586,Michael Folivi,1,6,0,0,0,0,0,0,0,0,S,Watford,0,0,0,0
522,5598,Angel Gomes,1,1,0,0,0,0,0,0,0,0,S,Manchester United,0,0,0,0
